Věrohodnost

In [31]:
import numpy as np
import pandas as pd 


import matplotlib.pyplot as plt
import scipy.optimize as optimize
import scipy
import seaborn as sns
import json
from zipfile import ZipFile

In [32]:
# load data
fileName = "Data_2024.xlsx"
df = pd.read_excel(fileName, sheet_name="Data_věrohodnost") # need pip install xlrd and openpyxl
df = df.iloc[:, :-2] # remove columns without information
# check types
df['censored'] = df['censored'].astype(int)
df['doba práce v oboru [roky]'] = df['doba práce v oboru [roky]'].astype(float)
print(df)

     censored  doba práce v oboru [roky]
0           0                      6.528
1           0                      6.013
2           1                      6.055
3           0                      7.243
4           1                      5.629
..        ...                        ...
316         1                      5.562
317         0                      5.491
318         0                      6.761
319         0                      7.062
320         0                      5.784

[321 rows x 2 columns]


f(x;lambda, k) = 

In [33]:
from scipy.optimize import minimize

def log_likelihood(params, x, censor):
    k, lamb = params
    return -np.sum(censor * (np.log(k) - np.log(lamb) + (k-1) * np.log(x/lamb) - (x/lamb)**k) + (1-censor) * (-(x/lamb)**k))
    #return - (n * np.log(k) - n*k*np.log(lamb) + (k-1)* np.sum(np.log(x)) - np.sum((x/lamb)**k))

init_guess = [1, 1]
    
# Define the bounds for the parameters
bounds = [(0, None), (0, None)]
    
x = df['doba práce v oboru [roky]'].values
censored = df['censored'].values

# Perform the optimization
res = minimize(log_likelihood, init_guess, args=(x, censored), method="SLSQP", bounds=bounds)

print(res)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 249.31509175824306
       x: [ 6.490e+00  8.641e+00]
     nit: 16
     jac: [-2.670e-05  6.866e-05]
    nfev: 61
    njev: 16


In [34]:
import surpyval as surv

# Failures
f = [1, 4, 5, 7, 8, 10]

# suspended or right-censored points
s = [5, 6, 7]

# Convert 'fs' format to the 'xcn' format 
# x is the time/variable
# c is the censoring flag 0 = failed, 1 = right-censored
# n is the count

x, c, n = surv.fs_to_xcn(f=f, s=s)
# Fit model
surv.Weibull.fit(x, c, n)

Parametric SurPyval Model
Distribution        : Weibull
Fitted by           : MLE
Parameters          :
     alpha: 7.738077941578665
      beta: 2.263120966029976

In [6]:
import surpyval as surv
values = df['doba práce v oboru [roky]'].values

# 0 = failed, 1 = right censored
censored = df['censored'].values

model = surv.Weibull.fit(values, c=censored)
print(model.params)

[7.42946141 6.17281648]


In [54]:
x = df['doba práce v oboru [roky]'].values

# 0 = failed, 1 = right censored
censored = df['censored'].values

# Logaritmická věrohodnost pro cenzorovaná data
def log_likelihood(params):
    k, lam = params
    if k <= 0 or lam <= 0:
        return np.inf  # Parametry musí být kladné
    # Hustota a přeživší funkce
    pdf = (k / lam) * (x / lam)**(k - 1) * np.exp(-(x / lam)**k)  # Hustota
    sf = np.exp(-(x / lam)**k)  # Přeživší funkce
    # Logaritmická věrohodnost
    log_l = (1 - censored) * np.log(pdf) + censored * np.log(sf)
    return -np.sum(log_l)  # Negace pro minimalizaci

# Počáteční odhady
initial_params = [1.0, 1.0]

# Optimalizace
result = minimize(log_likelihood, initial_params, method='L-BFGS-B', bounds=((1e-5, None), (1e-5, None)))

# Výsledky
if result.success:
    k_hat, lam_hat = result.x
    print(f"Odhadnuté parametry: k = {k_hat:.4f}, λ = {lam_hat:.4f}")
else:
    print("Optimalizace selhala:", result.message)

Odhadnuté parametry: k = 6.1728, λ = 7.4295


In [53]:
x = df['doba práce v oboru [roky]'].values

# 0 = failed, 1 = right censored
censored = df['censored'].values
# Negative log-likelihood with improved stability
def negative_log_likelihood(params):
    k, lam = params
    if k <= 0 or lam <= 0:  # Ensure positive parameters
        return np.inf

    # Uncensored contribution f(x) original 
    uncensored_loglik = np.log((k / lam) * (x[~censored] / lam) ** (k - 1) * np.exp(-(x[~censored] / lam) ** k))
    # Censored contribution (1- F(x)) log likelihood
    censored_loglik = np.log(1 - ((1/k) - np.log(lam) + np.log(x[censored]) - np.log(x[censored] / lam) * ((x[censored]/lam) ** k)))#(np.exp(-(x[censored] / lam) ** k)*(k/lam)*((x[censored]/lam)**(k-1))))

    # Combine and return negative log-likelihood
    nll = -(np.sum(uncensored_loglik) + np.sum(censored_loglik))
    return nll

initial_guess = [1, 1]
# Optimize with refined settings
result = minimize(negative_log_likelihood, initial_guess, method="L-BFGS-B")

# Display results
print(result)
k_mle, lambda_mle = result.x
print(f"Odhadnuté parametry: k = {k_mle:.8f}, lambda = {lambda_mle:.8f}")


  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: inf
        x: [ 1.000e+00  1.000e+00]
      nit: 0
      jac: [ 1.940e+03 -1.094e+03]
     nfev: 63
     njev: 21
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
Odhadnuté parametry: k = 1.00000000, lambda = 1.00000000


C:\Users\marti\AppData\Local\Temp\ipykernel_11856\4240214372.py:14: RuntimeWarning: invalid value encountered in log
  censored_loglik = np.log(1 - ((1/k) - np.log(lam) + np.log(x[censored]) - np.log(x[censored] / lam) * ((x[censored]/lam) ** k)))#(np.exp(-(x[censored] / lam) ** k)*(k/lam)*((x[censored]/lam)**(k-1))))
c:\Users\marti\anaconda3\envs\aidojo\Lib\site-packages\scipy\optimize\_numdiff.py:592: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


In [42]:
import numpy as np
from scipy.optimize import minimize

values = df['doba práce v oboru [roky]'].values

# 0 = failed, 1 = right censored
censored = df['censored'].values

# Definice negativní log-věrohodnostní funkce s cenzurou
def negative_log_likelihood(params):
    k, lam = params

    # Log-věrohodnost necenzurovaných dat
    uncensored_loglik = values[~censored] * (1 - np.log((k/lam) * (x/lam) ** (k-1) * np.exp(-(x/lam)**k) ))
    #uncensored_loglik = np.log((k / lam) * (values[~censored] / lam) ** (k - 1) *
    #                           np.exp(-(values[~censored] / lam) ** k))

    # Log-věrohodnost cenzurovaných dat
    censored_loglik = values[censored] * (np.log(k) - np.log(lam) * k + (k-1) * np.log(x) - (x/lam) **k)
    #censored_loglik = np.log(1 - np.exp(-(values[censored] / lam) ** k))

    # Celková negativní log-věrohodnost
    nll = -(np.sum(uncensored_loglik) + np.sum(censored_loglik))
    return nll

# Počáteční odhady parametrů
initial_guess = [1.0, 1.0]

# Optimalizace
result = minimize(negative_log_likelihood, initial_guess, method="L-BFGS-B", bounds=[(1e-6, None), (1e-6, None)])

# Výsledné odhady
k_mle, lambda_mle = result.x
print(f"Odhadnuté parametry: k = {k_mle:.4f}, lambda = {lambda_mle:.4f}")


ValueError: operands could not be broadcast together with shapes (321,) (9,) 

In [43]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import weibull_min

values = df['doba práce v oboru [roky]'].values

# 0 = failed, 1 = right censored
censored = df['censored'].values


# Definice negativní log-věrohodnostní funkce s cenzurou
def negative_log_likelihood(params):
    k, lam = params
    if k <= 0 or lam <= 0:  # parametry musí být kladné
        return np.inf

    # Log-věrohodnost necenzurovaných dat
    uncensored_loglik = np.log((k / lam) * (values[~censored] / lam) ** (k - 1) *
                               np.exp(-(values[~censored] / lam) ** k))

    # Log-věrohodnost cenzurovaných dat
    censored_loglik = np.log(1 - np.exp(-(values[censored] / lam) ** k))

    # Celková negativní log-věrohodnost
    nll = -(np.sum(uncensored_loglik) + np.sum(censored_loglik))
    return nll

# Počáteční odhady parametrů
initial_guess = [1.0, 1.0]

# Optimalizace
result = minimize(negative_log_likelihood, initial_guess, method="L-BFGS-B", bounds=[(1e-6, None), (1e-6, None)])

# Výsledné odhady
k_mle, lambda_mle = result.x
print(f"Odhadnuté parametry: k = {k_mle:.4f}, lambda = {lambda_mle:.4f}")


Odhadnuté parametry: k = 9.2078, lambda = 6.1292


In [ ]:
from scipy.optimize import minimize

def shapeDeriv():
    n/k - n * np.log(lamb) + np.sum(np.log(x)) - np.sum(np.log(x) - np.log(lamb)*((x/lamb)**k))

# Define the log-likelihood function
def log_likelihood(params, x, censored):
    k, lambda_ = params
    return np.sum(np.log(k) + (k-1) * np.log(x/lambda_) - (x/lambda_)**k) - np.sum((x/lambda_)**k * censored)

# Define the objective function
def objective(params, x, censored):
    return -log_likelihood(params, x, censored)

# Load the data
x = df['doba práce v oboru [roky]'].values
censored = df['censored'].values

# Initial guess for the shape and scale parameters
k0 = 1
lambda0 = 1

# Solve the system of equations using the minimize function from scipy
sol = minimize(objective, [k0, lambda0], args=(x, censored))

print("Shape parameter:", sol.x[0])
print("Scale parameter:", sol.x[1])

In [34]:
from scipy.optimize import minimize

# Define the log-likelihood function
def log_likelihood(params, x, censored):
    k, lambda_ = params
    return np.sum(np.log(k) + (k-1) * np.log(x/lambda_) - (x/lambda_)**k) - np.sum((x/lambda_)**k * censored)

# Define the objective function
def objective(params, x, censored):
    return -log_likelihood(params, x, censored)

# Load the data
x = df['doba práce v oboru [roky]'].values
censored = df['censored'].values

# Initial guess for the shape and scale parameters
k0 = 1
lambda0 = 1

# Solve the system of equations using the minimize function from scipy
sol = minimize(objective, [k0, lambda0], args=(x, censored))

print("Shape parameter:", sol.x[0])
print("Scale parameter:", sol.x[1])

Shape parameter: 6.094172961152544
Scale parameter: 7.590159395476218


In [12]:
# shape derivative
# np.log is ln
def shape(x, n, k, lamb):
    shape = (n/k) - n*np.log(lamb) + sum(np.log(x[i])) - sum((np.log(x[i]) - np.log(lamb)) * (x[i]/lamb)^k)
# scale derivative

In [11]:
# TODO read: https://real-statistics.com/distribution-fitting/distribution-fitting-via-maximum-likelihood/weibull-censored-data/
optimize.minimize(shape, df)

ValueError: 'x0' must only have one dimension.

In [19]:
import numpy as np
from scipy.optimize import minimize

def log_likelihood(params, x, censored):
    k, lambda_ = params
    return -np.sum(np.log(k/lambda_) + (k-1) * np.log(x/lambda_) - (x/lambda_)**k) - np.sum((x/lambda_)**k * censored)

def gradient(params, x, censored):
    k, lambda_ = params
    dLdk = np.sum(1/k + np.log(x/lambda_)) + np.sum(-(x/lambda_)**k * np.log(x/lambda_)) * censored
    dLdlambda = np.sum(-1/lambda_ + (k/lambda_) * (x/lambda_)**(k-1)) + np.sum((k/lambda_) * (x/lambda_)**(k-1)) * censored
    return np.array([dLdk, dLdlambda])

def objective(params, x, censored):
    return log_likelihood(params, x, censored)

# Načtěte data z Excelového souboru
import pandas as pd
data = pd.read_excel('Data_2024.xlsx', sheet_name='Data_věrohodnost')

x = df['doba práce v oboru [roky]'].values
censored = df['censored'].values

# Definujte počáteční odhady parametrů
params0 = np.array([1, 1])

# Minimizujte negativní logaritmickou-věrohodnostní funkci
res = minimize(objective, params0, args=(x, censored), method='SLSQP', jac=gradient)

# Vypište maximálně věrohodné odhady parametrů
print(res.x)

ValueError: _slsqp._slsqp.slsqp: failed to create array from the 8th argument `g` -- 0-th dimension must be fixed to 3 but got 643

In [17]:
x = df["doba práce v oboru [roky]"].values
d = df["censored"].values

def neg_log_likelihood(params, x, d):
    a, b = params
    u = x / b
    ll = -np.sum(d * (np.log(a/b) + (a-1)*np.log(u) - u*a) + (1-d) * np.log(1 - np.exp(-u*a)))
    return ll

res = scipy.optimize.minimize(neg_log_likelihood, x0=(1, 1), args=(x, d), method='Nelder-Mead')
a_hat, b_hat = res.x

print(f"Maximálně věrohodné odhady: shape={a_hat:.3f}, scale={b_hat:.3f}")

Maximálně věrohodné odhady: shape=1.564, scale=4.546


In [13]:
res = scipy.stats.weibull_min.sf(df)
print(res)

TypeError: _parse_args() missing 1 required positional argument: 'c'

In [14]:
res = scipy.stats.weibull_min(df['doba práce v oboru [roky]'])
print(res)

In [ ]:
# https://stackoverflow.com/questions/17481672/fitting-a-weibull-distribution-using-scipy
params = scipy.stats.exponweib.fit(df['doba práce v oboru [roky]'], floc=0, f0=1)
shape = params[1]
scale = params[3]
print ('shape:',shape)
print ('scale:',scale)

shape: 1.0
scale: 1.05


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import weibull_min
from scipy.optimize import minimize
from statsmodels.duration import api as smd
import statsmodels.api as sm

# ... (Load and prepare your data: 'time', 'event' columns) ...
x = df['doba práce v oboru [roky]'].values
censored = df['censored'].values

# Define the log-likelihood function for right-censored Weibull
def weibull_loglik(params, time, event):
    shape, scale = params
    loglik = 0
    for t, e in zip(time, event):
        if e == 1:  # Uncensored
            loglik += np.log(weibull_min.pdf(t, shape, scale=scale))
        else:  # Censored
            loglik += np.log(weibull_min.sf(t, shape, scale=scale))  # Survival function
    return -loglik  # Minimize the negative log-likelihood

# Initial parameter guess
initial_params = [1, 1]  # Shape, scale

# Minimize the negative log-likelihood
result = minimize(weibull_loglik, initial_params, args=(data['time'], data['event']), bounds=[(0.001, None), (0.001, None)]) # positive constraints on shape and scale.

# Extract estimated parameters
shape_est, scale_est = result.x
print(f"Estimated shape: {shape_est}, Estimated scale: {scale_est}")

KeyError: 'x'

In [3]:
scipy.stats.exponweib.fit(df, floc=0, f0=1)

(1, 1.0, 0, 1.05)

k shape parameter volím väčší ako 1 k > 1 pretože sa časom predpoklada, že menej ľudi bude časom pracovať vo vyštudovanom obore.

Regrese